<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/krosnicki/grocery%26moviesPUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd


ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')


total_ratings = ratings.shape[0]
total_movies = movies.shape[0]

total_users = ratings['userId'].nunique()

avg_ratings_per_user = total_ratings / total_users

avg_ratings_per_movie = total_ratings / total_movies


ratings_distribution = ratings['rating'].value_counts().sort_index()

# Wyświetlenie wyników
print("Liczba wszystkich ocen:", total_ratings)
print("Liczba filmów w bazie:", total_movies)
print("Liczba użytkowników:", total_users)
print("Średnia liczba ocen na użytkownika:", avg_ratings_per_user)
print("Średnia liczba ocen na film:", avg_ratings_per_movie)
print("\nRozkład ocen:")
print(ratings_distribution)


Liczba wszystkich ocen: 100836
Liczba filmów w bazie: 9742
Liczba użytkowników: 610
Średnia liczba ocen na użytkownika: 165.30491803278687
Średnia liczba ocen na film: 10.350646684459043

Rozkład ocen:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64


In [4]:
# Słownik
movie_titles = dict(zip(movies['movieId'], movies['title']))


C = ratings['rating'].mean()
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()

#bayesian_avg
def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg

#bayesian_avg_ratings
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']

movie_stats = movies.merge(bayesian_avg_ratings, on='movieId')

# Wyświetlenie wyników
print(movie_stats.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                                       Comedy      3.558189  


In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

total_ratings = ratings.shape[0]

total_movies = movies.shape[0]
total_users = ratings['userId'].nunique()


avg_ratings_per_user = total_ratings / total_users

avg_ratings_per_movie = total_ratings / total_movies

# Rozkład ocen
ratings_distribution = ratings['rating'].value_counts().sort_index()

# Wyświetlenie wyników
print("Liczba wszystkich ocen:", total_ratings)
print("Liczba filmów w bazie:", total_movies)
print("Liczba użytkowników:", total_users)
print("Średnia liczba ocen na użytkownika:", avg_ratings_per_user)
print("Średnia liczba ocen na film:", avg_ratings_per_movie)
print("\nRozkład ocen:")
print(ratings_distribution)

movie_titles = dict(zip(movies['movieId'], movies['title']))

C = ratings['rating'].mean()
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()

#bayesian_avg
def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg

#bayesian_avg_ratings
bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']


movie_stats = movies.merge(bayesian_avg_ratings, on='movieId')

# Wyświetlenie wyników
print(movie_stats.head())

def create_sparse(df):
    M = df['userId'].nunique()
    N = df['movieId'].nunique()
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]
    X = csr_matrix((df["rating"], (user_index, item_index)), shape=(M,N))
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_sparse(ratings)

n_total = X.shape[0] * X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings / n_total
sparsity_p = round(sparsity * 100, 2)

print(f"Matrix sparsity: {sparsity_p}%")


Liczba wszystkich ocen: 100836
Liczba filmów w bazie: 9742
Liczba użytkowników: 610
Średnia liczba ocen na użytkownika: 165.30491803278687
Średnia liczba ocen na film: 10.350646684459043

Rozkład ocen:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                  

In [6]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    X = X.T
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1, -1)
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    neighbour = neighbour[0][1:]
    for n in neighbour:
        neighbour_ids.append(movie_inv_mapper[n])
    return neighbour_ids

recommendations = find_similar_movies(1, X, movie_mapper, movie_inv_mapper, k=10)

# Wyświetlenie rekomendacji
for i, movie_id in enumerate(recommendations, 1):
    print(f"{i}. {movie_titles[movie_id]}")


1. Toy Story 2 (1999)
2. Jurassic Park (1993)
3. Independence Day (a.k.a. ID4) (1996)
4. Star Wars: Episode IV - A New Hope (1977)
5. Forrest Gump (1994)
6. Lion King, The (1994)
7. Star Wars: Episode VI - Return of the Jedi (1983)
8. Mission: Impossible (1996)
9. Groundhog Day (1993)
10. Back to the Future (1985)


In [7]:

!pip install surprise
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy


min_rating = ratings.rating.min()
max_rating = ratings.rating.max()
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD()

#cross-validation
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8810  0.8631  0.8701  0.8691  0.8813  0.8729  0.0071  
MAE (testset)     0.6738  0.6672  0.6685  0.6693  0.6762  0.6710  0.0034  
Fit time          2.25    1.61    1.84    1.58    1.56    1.76    0.26    
Test time         0.26    0.14    0.24    0.13    0.25    0.20    0.06    


{'test_rmse': array([0.88095237, 0.86308953, 0.87005302, 0.86908367, 0.88132632]),
 'test_mae': array([0.67375794, 0.66717734, 0.66851478, 0.66929015, 0.67622163]),
 'fit_time': (2.2495460510253906,
  1.6065635681152344,
  1.8352391719818115,
  1.5752620697021484,
  1.5573420524597168),
 'test_time': (0.25607967376708984,
  0.13532543182373047,
  0.24147367477416992,
  0.13227033615112305,
  0.250821590423584)}

In [8]:

trainset, testset = train_test_split(data, test_size=0.25)

algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)

RMSE: 0.8800


0.8799578908510743

KNN

In [13]:
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Reader
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split
lgo = KNNBasic()
algo.fit(trainset)
predict = algo.test(testset)
accuracy.rmse(predict)


RMSE: 0.8792


0.8792029214760289

In [15]:
param_grid = {
    'k': [1, 2, 5, 10, 20, 30, 40, 50]
}

gs_knn = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs_knn.fit(data)

In [16]:
gs_knn.best_params

cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9451  0.9461  0.9411  0.9550  0.9472  0.9469  0.0045  
MAE (testset)     0.7256  0.7256  0.7231  0.7299  0.7258  0.7260  0.0022  
Fit time          0.11    0.14    0.14    0.15    0.17    0.14    0.02    
Test time         1.39    1.36    1.53    1.53    2.31    1.62    0.35    


{'test_rmse': array([0.94513791, 0.94607912, 0.94111887, 0.95501451, 0.94715329]),
 'test_mae': array([0.72562038, 0.72557226, 0.72313436, 0.72994282, 0.72578605]),
 'fit_time': (0.10595035552978516,
  0.13922476768493652,
  0.1414659023284912,
  0.14609527587890625,
  0.17458367347717285),
 'test_time': (1.3925535678863525,
  1.3624820709228516,
  1.5277931690216064,
  1.5336651802062988,
  2.306063652038574)}

In [17]:
gs_knn.best_params

{'rmse': {'k': 20}}

In [18]:
algo = KNNBaseline()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predict)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8792


0.8792029214760289